In [1]:
pip install gradio


Note: you may need to restart the kernel to use updated packages.


In [15]:

import gradio as gr
import numpy as np
from PIL import Image
from tensorflow.keras.models import load_model

# Load your model
model = load_model("deepfake_model.keras")

# Preprocessing function
def preprocess(img):
    img = img.resize((96, 96))
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

# Prediction function that returns (image, label) pairs
def predict(files):
    results = []
    for file in files:
        image = Image.open(file.name).convert("RGB")
        x = preprocess(image)
        prob = model.predict(x)[0][0]
        label = "REAL" if prob >= 0.5 else "FAKE"
        confidence = prob if label == "REAL" else 1 - prob
        text = f"{label} ({confidence:.2%} confident)"
        results.append((image, text))
    return results

# Gradio interface with Gallery output (no .style())
interface = gr.Interface(
    fn=predict,
    inputs=gr.File(file_types=["image"], file_count="multiple", label="Upload Multiple Images"),
    outputs=gr.Gallery(label="Predictions"),
    title="Deepfake Detector",
    description="Upload multiple image files (frames) to detect if they are real or fake."
)

interface.launch(inline=False, server_port=7890)




* Running on local URL:  http://127.0.0.1:7890

To create a public link, set `share=True` in `launch()`.


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
